### Data exploratory analysis

In [ ]:
url = r"https://raw.githubusercontent.com/MSI17819/Berlin_bike_theft_prediction/main/Berlin_LOR_code.csv"

df = pd.read_csv(url, sep=';', encoding="UTF-8")

df.info()

In [ ]:
df.groupby('Record_day_of_week')[['Record_date']].count().sort_values(by='Record_date', ascending=False)

In [ ]:
df.groupby('Record_day_of_week')[['Record_date']].count().sort_values(by='Record_date', ascending=True).plot(kind='barh',
                                                                                                                   figsize=(12,10),
                                                                                                                   fontsize=12,
                                                                                                                   title='Amount of crimes by Day of Week',
                                                                                                                   xlabel='Day of week',
                                                                                                                   grid=True)

In [ ]:
df.groupby('Administrative_district')[['Record_date']].count().sort_values(by=['Record_date'], ascending=False)

In [ ]:
df.groupby('Administrative_district')[['Record_date']].count().sort_values(by=['Record_date'], ascending=True).plot(kind='barh',
                                                                                                                          figsize=(12,10),
                                                                                                                          grid=True)

In [ ]:
df.groupby('Bike_type')[['Record_date']].count().sort_values(by='Record_date', ascending=False)

In [ ]:
df.groupby('Crime_type')[['Record_date']].count().sort_values(by='Record_date', ascending=False)